In [9]:
import psycopg2
from psycopg2 import OperationalError
import requests

In [7]:
def create_table(
    dbname: str,
    user: str,
    password: str,
    host: int,
    port: int
) -> None:

    conn_params = {
        'dbname': dbname,
        'user': user,
        'password': password,
        'host': host,
        'port': port
    }
    
    # Connect to your postgres DB
    conn = psycopg2.connect(conn_params)

    cur = conn.cursor()

    # Define the CREATE TABLE statement
    create_table_query = '''
    CREATE TABLE demand (
        id SERIAL PRIMARY KEY,
        period VARCHAR(100),
        subba VARCHAR(100),
        subba_name VARCHAR(100),	
        parent VARCHAR(100),	
        parent_name VARCHAR(100),	
        timezone VARCHAR(100),	
        value INTEGER,	
        value_units VARCHAR(100)
    );
    '''

    # Execute the query
    cur.execute(create_table_query)

    # Commit the changes
    conn.commit()

    # Close the cursor and connection
    cur.close()
    conn.close()

In [40]:
def generate_url(
    start_date, 
    end_date, 
    offset, 
    chunk_len, 
    api_key,
    use_api
):
    url_demand = f'https://api.eia.gov/v2/electricity/rto/' +\
    f'daily-region-sub-ba-data/data/?frequency=daily&data[0]=value' + \
    f'&facets[subba][]=ZONJ&start={start_date}&end={end_date}' + \
    f'&sort[0][column]=period&sort[0][direction]=desc&offset={offset}' + \
    f'&length={chunk_len}&api_key={api_key}'

    url_weather_historical = f'https://archive-api.open-meteo.com/v1' + \
    f'/archive?latitude=52.52&longitude=13.41&start_date={start_date}' + \
    f'&end_date={end_date}&daily=weathercode,temperature_2m_max,' + \
    f'temperature_2m_min,temperature_2m_mean' + \
    f'&timezone=America%2FNew_York'
    
    url_weather_latest = f'https://api.open-meteo.com/v1/forecast?' + \
    f'latitude=52.52&longitude=13.41&hourly=temperature_2m&daily=' + \
    f'temperature_2m_max,temperature_2m_min&timezone=America%2F' + \
    f'New_York&past_days=7'

    api_urls = {
        'eia': url_demand,
        'weather_hist': url_weather_historical,
        'weather_latest': url_weather_latest
    }

    return api_urls.get(use_api, None)

In [43]:
start_date = '2018-01-01'
end_date = '2022-07-31'
offset = 0
chunk_len = 10000
api_key = ''
use_api = 'weather_latest'

url = generate_url(start_date, end_date, offset, chunk_len, api_key, use_api)
url

'https://api.open-meteo.com/v1/forecast?latitude=52.52&longitude=13.41&hourly=temperature_2m&daily=temperature_2m_max,temperature_2m_min&timezone=America%2FNew_York&past_days=7'

In [8]:
def get_data(url):
    response = requests.get(url)

    if response.status_code == 200:  # Success
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

In [ ]:
def main():
    
    return